This example makes a reactor geometry and a neutronics model. A homogenised material made of enriched lithium lead and eurofer is being used as the blanket material for this simulation in order to demonstrate the use of more complex materials.

In [ ]:
import neutronics_material_maker as nmm
import openmc
import paramak
from parametric_plasma_source import SOURCE_SAMPLING_PATH, PlasmaSource


# makes the 3d geometry
my_reactor = paramak.BallReactor(
    inner_bore_radial_thickness=10,
    inboard_tf_leg_radial_thickness=30,
    center_column_shield_radial_thickness=60,
    divertor_radial_thickness=150,
    inner_plasma_gap_radial_thickness=30,
    plasma_radial_thickness=300,
    outer_plasma_gap_radial_thickness=30,
    firstwall_radial_thickness=30,
    blanket_radial_thickness=50,
    blanket_rear_wall_radial_thickness=30,
    elongation=2,
    triangularity=0.55,
    number_of_tf_coils=16,
    pf_coil_case_thicknesses=[10, 10, 10, 10],
    pf_coil_radial_thicknesses=[20, 50, 50, 20],
    pf_coil_vertical_thicknesses=[20, 50, 50, 20],
    pf_coil_radial_position=[500, 575, 575, 500],
    pf_coil_vertical_position=[300, 100, -100, -300],
    rear_blanket_to_tf_gap=50,
    outboard_tf_coil_radial_thickness=100,
    outboard_tf_coil_poloidal_thickness=50,
    rotation_angle=180,
)

# method is set to Trelis or Cubit by default to avoid overlaps in the geometry
# pymoab is used as it is open source and can be tested in the CI
# if you have Trelis or Cubit then this line can be deleted
my_reactor.method='pymoab'

my_reactor.show()

In [ ]:
# creates a parametric plasma source, more details
# https://github.com/open-radiation-sources/parametric-plasma-source
my_plasma = PlasmaSource(
    ion_density_origin=1.09e20,
    ion_density_peaking_factor=1,
    ion_density_pedestal=1.09e20,
    ion_density_separatrix=3e19,
    ion_temperature_origin=45.9,
    ion_temperature_peaking_factor=8.06,
    ion_temperature_pedestal=6.09,
    ion_temperature_separatrix=0.1,
    elongation=2,
    triangularity=0.55,
    major_radius=2.5,  # note the source takes m arguments
    minor_radius=1.,  # note the source takes m arguments
    pedestal_radius=0.8 * 100,  # note the source takes m arguments
    plasma_id=1,
    shafranov_shift=0.44789,
    ion_temperature_beta=6
)

# assigns parametric plasma source as source
source = openmc.Source()
source.library = SOURCE_SAMPLING_PATH
source.parameters = str(my_plasma)

In [ ]:
# makes the neutronics material
neutronics_model = paramak.NeutronicsModel(
    geometry=my_reactor,
    source=source,
    materials={
        'inboard_tf_coils_mat': 'copper',
        'center_column_shield_mat': 'WC',
        'divertor_mat': 'eurofer',
        'firstwall_mat': 'eurofer',
        'blanket_mat': 'eurofer',
        'blanket_rear_wall_mat': 'eurofer'},
)

In [ ]:
# exports the geometry and source in 2d (RZ) viewplane where R stands for
# radius
neutronics_model.export_html(
    filename='2d_source.html',
    view_plane='RZ'
)

# exports the geometry and source in 3d (XYZ) viewplane
neutronics_model.export_html(
    filename='3d_source.html',
    view_plane='XYZ',
    number_of_source_particles=1000
)